# White-wine data reading and cleaning

In [530]:
#Import and read the white-wine file which is in CSV format using pandas.
import pandas as pd
df=pd.read_csv(r"C:\Users\varsh\OneDrive\Desktop\MSDA\SEM2\MACHINE_LEARNING\Project\datasets\winequality-white.csv",sep=';',header='infer')
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [531]:
#checking and renaming headers into single value.
df=df.rename(columns={'fixed acidity': 'fixed_acidity','volatile acidity': 'volatile_acidity', 'citric acid': 'citric_acid','free sulfur dioxide': 'free_sulfur_dioxide','residual sugar': 'residual_sugar','total sulfur dioxide': 'total_sulfur_dioxide'})
df

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [532]:
#checking for categorical values.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         4898 non-null   float64
 1   volatile_acidity      4898 non-null   float64
 2   citric_acid           4898 non-null   float64
 3   residual_sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free_sulfur_dioxide   4898 non-null   float64
 6   total_sulfur_dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [533]:
#Checking and removal of duplicates, if any present.
print("The number of non-distinct data is :",df.count())
df=df.drop_duplicates()
print("After dropping the duplicates, the count of the dictinct data is:",df.count()) 

The number of non-distinct data is : fixed_acidity           4898
volatile_acidity        4898
citric_acid             4898
residual_sugar          4898
chlorides               4898
free_sulfur_dioxide     4898
total_sulfur_dioxide    4898
density                 4898
pH                      4898
sulphates               4898
alcohol                 4898
quality                 4898
dtype: int64
After dropping the duplicates, the count of the dictinct data is: fixed_acidity           3961
volatile_acidity        3961
citric_acid             3961
residual_sugar          3961
chlorides               3961
free_sulfur_dioxide     3961
total_sulfur_dioxide    3961
density                 3961
pH                      3961
sulphates               3961
alcohol                 3961
quality                 3961
dtype: int64


In [534]:
#checking for nulls or missing values.
df.isnull()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,False,False,False,False,False,False,False,False,False,False,False,False
4894,False,False,False,False,False,False,False,False,False,False,False,False
4895,False,False,False,False,False,False,False,False,False,False,False,False
4896,False,False,False,False,False,False,False,False,False,False,False,False


In [535]:
#Create a user defined function to convert the interger values of 'quality' column into binary values such as 0 and 1.
import numpy as np
df['quality'] = np.where(df['quality']>=6, 1, 0)

C:\Users\varsh\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [536]:
df

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,1
6,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.99490,3.18,0.47,9.6,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,1
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,0
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,1
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,1


In [537]:
#Seperating X and y values and storing in different variables.
X = df[['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol']]
y=df['quality']

In [538]:
#Standardize the feature-vector values.
from sklearn.preprocessing import StandardScaler
sclr=StandardScaler()
X = sclr.fit_transform(X)
X

array([[ 0.18535197, -0.10188873,  0.20965146, ..., -1.28992816,
        -0.35548818, -1.47039574],
       [-0.6222624 ,  0.18817926,  0.04629365, ...,  0.68992438,
        -0.00309159, -0.89517431],
       [ 1.45446025, -0.0051994 ,  0.53636707, ...,  0.42594404,
        -0.44358733, -0.40212736],
       ...,
       [-0.39151544, -0.39195672, -1.17888992, ..., -1.35592325,
        -0.26738904, -0.9773488 ],
       [-1.54525024,  0.09148993, -0.28042197, ...,  0.95390472,
        -0.97218223,  1.81658387],
       [-0.96838284, -0.6820247 ,  0.37300927, ...,  0.42594404,
        -1.50077712,  0.99483897]])

In [539]:
#split the data into training and test data.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=1)
print("The number of rows and columns in the training dataset post split is:",X_train.shape)
print("The number of rows and columns in the training dataset post split is:",y_train.shape)
print("The number of rows and columns in the test dataset post split is:",X_test.shape)
print("The number of rows and columns in the tes dataset post split is:",y_test.shape)

The number of rows and columns in the training dataset post split is: (2772, 11)
The number of rows and columns in the training dataset post split is: (2772,)
The number of rows and columns in the test dataset post split is: (1189, 11)
The number of rows and columns in the tes dataset post split is: (1189,)


# White-wine Regression 

In [540]:
#create a decision tree regression model using the training dataset.
from sklearn import tree
decisionTreeRegressor = tree.DecisionTreeRegressor()
DTRegModel = decisionTreeRegressor.fit(X_train, y_train)
DTRegModel

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [541]:
#Test the model with the test dataset for predictions (without the labels).
DTReg_predictions=DTRegModel.predict(X_test)
DTReg_predictions

array([0., 0., 0., ..., 0., 0., 0.])

In [542]:
#hyper parameter tuning for a better accuracy of our decision tree model.
parameter_dist={'criterion':['mse','mae'], 
                'max_depth':[1,2,3,4,5,6,7,None]}

In [ ]:
#one of the method used for fine tuning the algorithm.
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(decisionTreeRegressor,param_grid=parameter_dist,cv=10,n_jobs=-1)
grid.fit(X_train,y_train)

In [ ]:
#determine best parameter values of a decision tree.
grid.best_estimator_

In [ ]:
#determine the best accuracy score.
print(f'The score is: {round((grid.best_score_*100),2)}%')

In [ ]:
#determine the best parameters for fine tuning.
grid.best_params_

In [ ]:
#hyper tuning the parameters and training the model.
decisionTreeRegressor = tree.DecisionTreeRegressor(criterion= 'mse', max_depth= 3)
DTRegModel = decisionTreeRegressor.fit(X_train, y_train)
DTRegModel

In [ ]:
#Test the model with the test dataset for predictions (without the labels).
DTReg_predictions=DTRegModel.predict(X_test)
DTReg_predictions

In [ ]:
#Check mean absolute error value.
from sklearn.metrics import mean_absolute_error
MAE=mean_absolute_error(y_test, DTReg_predictions)
print(f'The mean absolute error of decision tree regression for white-wine quality prediction is {round((MAE*100),2)}%')


In [ ]:
#check the mean squared error value.
from sklearn.metrics import mean_squared_error
MSE=mean_squared_error(y_test, DTReg_predictions,squared=False)
print(f'The mean squared error of decision tree regression for white-wine quality prediction is {round((MSE*100),2)}%')


In [ ]:
#Check the root mean squared error (RMSE) value.
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test, DTReg_predictions,squared=True)
print(f'The root mean squared error of decision tree regression for white-wine quality prediction is {round((RMSE*100),2)}%')

In [ ]:
#Check the regression score function value (r2-score).
from sklearn.metrics import r2_score
r2_score=r2_score(y_test, DTReg_predictions)
print(f'The regression score function value of decision tree regression for white-wine quality prediction is {round((r2_score*100),2)}%')

In [ ]:
#plot the decision tree for white-wine dataset
from matplotlib import pyplot as plt
plt.figure(figsize=(15,10))
tree.plot_tree(decisionTreeRegressor,filled=True)

In [ ]:
#Division of tree shown in the form of text.
print(tree.export_text(decisionTreeRegressor))

# White-wine Classification 

In [ ]:
#create a decision tree classification model using the training dataset.
decisionTreeClassifier=tree.DecisionTreeClassifier()
DTClsModel=decisionTreeClassifier.fit(X_train,y_train)
DTClsModel

In [ ]:
#Test the model with the test dataset for predictions(without the labels).
DTClass_predictions=DTClsModel.predict(X_test)
DTClass_predictions

In [ ]:
#check the probability of  predicted labels.
DTClsModel.predict_proba(X_test)

In [ ]:
#Check the accuracy of decision tree classifier before fine tuning.
from sklearn.metrics import accuracy_score
accuracy_score=accuracy_score(y_test, DTClass_predictions)
print(f'The accuracy  of decision tree classification for white-wine quality prediction is {round((accuracy_score*100),2)}%')


In [ ]:
#hyper parameter tuning for a better accuracy of our decision tree model.
parameter_dist={'criterion':['gini','entropy'], 
                'max_depth':[1,2,3,4,5,6,7,None]}


In [ ]:
#one of the method use for fine tuning the algorithm.
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(decisionTreeClassifier,param_grid=parameter_dist,cv=10,n_jobs=-1)
grid.fit(X_train,y_train)


In [ ]:
#determine best parameter values of a decision tree.
grid.best_estimator_


In [ ]:
#determine the best accuracy score.
print(f'The score is: {round((grid.best_score_*100),2)}%')

In [ ]:
#determine the best parameters for fine tuning.
grid.best_params_


In [ ]:
#hyper tuning the parameters and training the model.
decisionTreeClassifier = tree.DecisionTreeClassifier(criterion= 'entropy', max_depth= 3)
DTClsModel = decisionTreeClassifier.fit(X_train, y_train)
DTClsModel


In [ ]:
#Test the model with the test dataset for predictions (without the labels).
DTClass_predictions=DTClsModel.predict(X_test)
DTClass_predictions


In [ ]:
#check the probability of top 5 predicted labels.
prob=DTClsModel.predict_proba(X_test)
print("The probability is :",prob[:5,:])

In [ ]:
#Check the balanced accuracy score to determine the imbalanced data prediction.
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score=balanced_accuracy_score(y_test, DTClass_predictions)
print(f'The balanced score accuracy  of decision tree regression for white-wine quality prediction is {round((balanced_accuracy_score*100),2)}%')

In [ ]:
#Check the accuracy of decision tree classifier post fine tuning.
from sklearn.metrics import accuracy_score
accuracy_score=accuracy_score(y_test, DTClass_predictions)
print(f'The accuracy  of decision tree classification for white-wine quality prediction is {round((accuracy_score*100),2)}%')


In [ ]:
#Check the precision score of decision tree classifier.
from sklearn.metrics import precision_score
precision_score=precision_score(y_test, DTClass_predictions)
print(f'The precision score of decision tree classification for white-wine quality prediction is {round((precision_score*100),2)}%')

In [ ]:
#Check the recall score of decision tree classifier.
from sklearn.metrics import recall_score
recall_score=recall_score(y_test, DTClass_predictions)
print(f'The recall score  of decision tree classification for white-wine quality prediction is {round((recall_score*100),2)}%')

In [ ]:
#Check the F1-score of decision tree classifier.
from sklearn.metrics import f1_score
f1_score=f1_score(y_test, DTClass_predictions)
print(f'The f1-score of decision tree classification for white-wine quality prediction is {round((f1_score*100),2)}%')

In [ ]:
# Plot the ROC curve.
import matplotlib.pyplot as plt  
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, DTClass_predictions)
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,estimator_name='example estimator')
display.plot(color='orange')  
plt.show()   

In [ ]:
# Plot the precision-recall curve.
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, DTClass_predictions)
print('The Average precision-recall score is {0:0.2f}%'.format(((average_precision*100))))

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
display = plot_precision_recall_curve(DTClsModel, X_test, y_test)
display.ax_.set_title('The Precision-Recall curve: ''AP={0:0.2f}%'.format(((average_precision*100))))

In [ ]:
# determine Receiver Operating Characteristic Curve (ROC AUC) score.
from sklearn.metrics import roc_auc_score
roc_auc_score = roc_auc_score(y_test, DTClass_predictions)
print('The Receiver Operating Characteristic Curve (ROC AUC) score is {0:0.2f}%'.format(((roc_auc_score*100))))

In [ ]:
#determine the classifcation report.
from sklearn.metrics import classification_report
print(classification_report(y_test, DTClass_predictions))

In [ ]:
#build the confusion matrix.
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
confusion_matrix(y_test, DTClass_predictions)

In [ ]:
#plot the confusion metrics.
display = plot_confusion_matrix(DTClsModel, X_test, y_test,display_labels=y,cmap=plt.cm.Greens)
display.ax_.set_title('Confusion matrix')
print(display.confusion_matrix)


In [ ]:
#plot the decision tree for white-wine dataset.
plt.figure(figsize=(15,10))
tree.plot_tree(decisionTreeClassifier,filled=True)


In [ ]:
#Division of tree shown in the form of text.
print(tree.export_text(decisionTreeClassifier))


# Red-wine data reading and cleaning

In [ ]:
#Import and read the red-wine file which is in CSV format using pandas.
import pandas as pd
df=pd.read_csv(r"C:\Users\varsh\OneDrive\Desktop\MSDA\SEM2\MACHINE_LEARNING\Project\datasets\winequality-red.csv",sep=',',header='infer')
df


In [ ]:
#checking and renaming headers into single value.
df=df.rename(columns={'fixed acidity': 'fixed_acidity','volatile acidity': 'volatile_acidity', 'citric acid': 'citric_acid','free sulfur dioxide': 'free_sulfur_dioxide','residual sugar': 'residual_sugar','total sulfur dioxide': 'total_sulfur_dioxide'})
df


In [ ]:
#checking for categorical values.
df.info()


In [ ]:
#Checking and removal of duplicates, if any present.
print("The number of non-distinct data is :",df.count())
df=df.drop_duplicates()
print("After dropping the duplicates, the count of the dictinct data is:",df.count())

In [ ]:
#checking for nulls or missing values.
df.isnull()


In [ ]:
#Create a user defined function to convert the interger values of 'quality' column into binary values such as 0 and 1.
import numpy as np
df['quality'] = np.where(df['quality']>=6, 1, 0)
df

In [ ]:
#Seperating X and y values and storing in different variables.
X = df[['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol']]
y=df['quality']


In [ ]:
#Standardize the feature-vector values.
from sklearn.preprocessing import StandardScaler
sclr=StandardScaler()
X = sclr.fit_transform(X)
X


In [ ]:
#split the data into training and test data.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print("The number of rows and columns in the training dataset post split is:",X_train.shape)
print("The number of rows and columns in the training dataset post split is:",y_train.shape)
print("The number of rows and columns in the test dataset post split is:",X_test.shape)
print("The number of rows and columns in the tes dataset post split is:",y_test.shape)


# Red-wine regression

In [ ]:
#create a decision tree regression model using the training dataset.
from sklearn import tree
decisionTreeRegressor = tree.DecisionTreeRegressor()
DTRegModel = decisionTreeRegressor.fit(X_train, y_train)


In [ ]:
#Test the model with the test dataset for predictions (without the labels).
DTReg_predictions=DTRegModel.predict(X_test)
DTReg_predictions[:5]


In [ ]:
#hyper parameter tuning for a better accuracy of our decision tree model.
parameter_dist={'criterion':['mse','mae'], 
                'max_depth':[1,2,3,4,5,6,7,None]}


In [ ]:
#one of the method use for fine tuning the algorithm.
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(decisionTreeRegressor,param_grid=parameter_dist,cv=10,n_jobs=-1)
grid.fit(X_train,y_train)


In [ ]:
#determine best parameter values of a decision tree.
grid.best_estimator_


In [ ]:
#determine the best accuracy score.
print(f'The score is: {round((grid.best_score_*100),2)}%')


In [ ]:
#determine the best parameters for fine tuning.
grid.best_params_


In [ ]:
#hyper tuning the parameters and training the model.
decisionTreeRegressor = tree.DecisionTreeRegressor(criterion= 'mse', max_depth= 4)
DTRegModel = decisionTreeRegressor.fit(X_train, y_train)
DTRegModel



In [ ]:
#Test the model with the test dataset for predictions of top 5 labels (without the labels).
DTReg_predictions=DTRegModel.predict(X_test)
DTReg_predictions[:5]

In [ ]:
#check the mean absolute error value.
from sklearn.metrics import mean_absolute_error
MAE=mean_absolute_error(y_test, DTReg_predictions)
print(f'The mean absolute error score of decision tree classification for red-wine quality prediction is {round((MAE*100),2)}%')



In [ ]:
#check the mean squared error value.
from sklearn.metrics import mean_squared_error
MSE=mean_squared_error(y_test, DTReg_predictions,squared=False)
print(f'The mean squared error score of decision tree classification for red-wine quality prediction is {round((MSE*100),2)}%')


In [ ]:
#Check the root mean squared error (RMSE) value.
from sklearn.metrics import mean_squared_error
RMSE=mean_squared_error(y_test, DTReg_predictions,squared=True)
print(f'The root mean squared error score of decision tree classification for red-wine quality prediction is {round((RMSE*100),2)}%')


In [ ]:
#Check the regression score function value (r2-score).
from sklearn.metrics import r2_score
r2_score=r2_score(y_test, DTReg_predictions)
print(f'The regression score of decision tree classification for red-wine quality prediction is {round((r2_score*100),2)}%')

In [ ]:
#plot the decision tree for red-wine dataset
plt.figure(figsize=(15,10))
tree.plot_tree(decisionTreeRegressor,filled=True)

In [ ]:
#Division of tree shown in the form of text.
print(tree.export_text(decisionTreeRegressor))


# Red-wine Classification

In [ ]:
#create a decision tree classification model using the training dataset.
decisionTreeClassifier=tree.DecisionTreeClassifier()
DTClsModel=decisionTreeClassifier.fit(X_train,y_train)


In [ ]:
#Test the model with the test dataset for predictions for top 5 labels(without the labels).
DTClass_predictions=DTClsModel.predict(X_test)
DTClass_predictions[:5]

In [ ]:
#check the probability of top 5predicted labels.
prob=DTClsModel.predict_proba(X_test)
prob[:5]

In [ ]:
#Check the accuracy of decision tree classifier before fine tuning.
from sklearn.metrics import accuracy_score
accuracy_score=accuracy_score(y_test, DTClass_predictions)
print(f'The accuracy  of decision tree classification for red-wine quality prediction is {round((accuracy_score*100),2)}%')

In [ ]:
#hyper parameter tuning for a better accuracy of our decision tree model.
parameter_dist={'criterion':['gini','entropy'], 
                'max_depth':[1,2,3,4,5,6,7,None]}


In [ ]:
#one of the method use for fine tyning the alforithm
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(decisionTreeClassifier,param_grid=parameter_dist,cv=10,n_jobs=-1)
grid.fit(X_train,y_train)


In [ ]:
#determine best parameter values of a decision tree
grid.best_estimator_


In [ ]:
#determine the best accuracy score
print(f'The score is: {round((grid.best_score_*100),2)}%')


In [ ]:
#determine the best parameters for fine tuning
grid.best_params_


In [ ]:
#hyper tuning the parameters and training the model:
decisionTreeClassifier = tree.DecisionTreeClassifier(criterion= 'entropy', max_depth= 3)
DTClsModel = decisionTreeClassifier.fit(X_train, y_train)
DTClsModel


In [ ]:
#Test the model with the test dataset for predictions (without the labels).
DTClass_predictions=DTClsModel.predict(X_test)
DTClass_predictions[:5]


In [ ]:
#check the probability of top 5predicted labels.
prob=DTClsModel.predict_proba(X_test)
prob[:5]

In [ ]:
#Check the accuracy of decision tree classifier post fine tuning.
from sklearn.metrics import accuracy_score
accuracy_score=accuracy_score(y_test, DTClass_predictions)
print(f'The accuracy  of decision tree classification for red-wine quality prediction is {round((accuracy_score*100),2)}%')


In [ ]:
#Check the precision score of decision tree classifier.
from sklearn.metrics import precision_score
precision_score=precision_score(y_test, DTClass_predictions)
print(f'The precision score of decision tree classification for red-wine quality prediction is {round((precision_score*100),2)}%')


In [ ]:
#Check the recall score of decision tree classifier.
from sklearn.metrics import recall_score
recall_score=recall_score(y_test, DTClass_predictions)
print(f'The recall score  of decision tree classification for red-wine quality prediction is {round((recall_score*100),2)}%')


In [ ]:
#Check the F1-score of decision tree classifier.
from sklearn.metrics import f1_score
f1_score=f1_score(y_test, DTClass_predictions)
print(f'The f1-score of decision tree classification for red-wine quality prediction is {round((f1_score*100),2)}%')

In [ ]:
# Determine the ROC curve.
import matplotlib.pyplot as plt  
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_test, DTClass_predictions)
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,estimator_name='example estimator')
display.plot(color='purple')  
plt.show()   


In [ ]:
# Plot the precision-recall curve.

from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test, DTClass_predictions)
print('The Average precision-recall score is {0:0.2f}%'.format(round((average_precision*100),2)))


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
display = plot_precision_recall_curve(DTClsModel, X_test, y_test)
display.ax_.set_title('The Precision-Recall curve: ''AP={0:0.2f}%'.format(round((average_precision*100),2)))

In [ ]:
# determine Receiver Operating Characteristic Curve (ROC AUC) score.
from sklearn.metrics import roc_auc_score
roc_auc_score = roc_auc_score(y_test, DTClass_predictions)
print('The Receiver Operating Characteristic Curve (ROC AUC) score is {0:0.2f}%'.format(((roc_auc_score*100))))

In [ ]:
#determine the classifcation report.
from sklearn.metrics import classification_report
print(classification_report(y_test, DTClass_predictions))

In [ ]:
#build the confusion matrix.
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
confusion_matrix(y_test, DTClass_predictions)


In [ ]:
#plot the confusion metrics
display = plot_confusion_matrix(DTClsModel, X_test, y_test,display_labels=y,cmap=plt.cm.Greens)
display.ax_.set_title('Confusion matrix')
print(display.confusion_matrix)

In [ ]:
#plot the decision tree for red-wine dataset
plt.figure(figsize=(15,10))
tree.plot_tree(decisionTreeClassifier,filled=True)

In [ ]:
#Division of tree shown in the form of text.
print(tree.export_text(decisionTreeClassifier))